## Load Llama-3-8B-SFT

In [1]:
# Warning: Using transformer version in DPO will lead to errors of loading Llama3
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
transformers.logging.set_verbosity_error()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("<some path>/model_zoo/Meta-Llama-3-8B",device_map="auto")
model = AutoModelForCausalLM.from_pretrained("<some path>/model_zoo/Meta-Llama-3-8B",device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
tokenizer_sft = AutoTokenizer.from_pretrained("<some path>/trl/models/Llama3_sft_UltraChat_SimPO/checkpoint-4872",device_map="auto")
model_sft = AutoModelForCausalLM.from_pretrained("<some path>/trl/models/Llama3_sft_UltraChat_SimPO/checkpoint-4872",device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
tokenizer_instruct = AutoTokenizer.from_pretrained("<some path>/model_zoo/Meta-Llama-3-8B-Instruct",device_map="auto")
model_instruct = AutoModelForCausalLM.from_pretrained("<some path>/model_zoo/Meta-Llama-3-8B-Instruct",device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
tokenizer_dpo = AutoTokenizer.from_pretrained("<some path>/trl/models/Llama3-Instruct_UltraFeedback_dpo",device_map="auto")
model_dpo = AutoModelForCausalLM.from_pretrained("<some path>/trl/models/Llama3-Instruct_UltraFeedback_dpo",device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
tokenizer_sft.chat_template

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

## DataSet Checking

In [7]:
from datasets import load_from_disk
dataset_path = "<some path>/trl/examples/datasets/UltraFeedback_trl"
ds = load_from_disk(dataset_path)

In [45]:
index = 50

input = ds["test"][index]["chosen"][:-1]

dash_line = "-".join("" for x in range(100))

print(dash_line)

print(tokenizer_dpo.apply_chat_template(input,tokenize=False))

print(dash_line)

dataset_chosen_response = ds["test"][index]["chosen"][-1:]

print(f"dataset_response (chosen):\n \n {tokenizer_dpo.apply_chat_template(dataset_chosen_response,tokenize=False)} ")


---------------------------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Teacher:In this task, you're given reviews from Amazon's food products and a summary of that review. Your task is to classify whether the given summary matches the original review. Generate "True" if the given review and its summary match, otherwise generate "False".
Teacher: Now, understand the problem? Solve this instance: Reviews: product was shipped in orginal box placed inside a larger box, no packing was used. Bulbs were broken when they arrived. 
 Summary: product was shipped in orginal box placed inside a larger ...
Student:<|eot_id|>
---------------------------------------------------------------------------------------------------
dataset_response (chosen):
 
 <|begin_of_text|><|start_header_id|>assistant<|end_header_id|>

True<|eot_id|> 


In [37]:
from transformers import pipeline
import torch

#model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer_sft,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    # device="cuda",
)

pipe_sft = pipeline(
    "text-generation",
    model=model_sft,
    tokenizer=tokenizer_sft,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    # device="cuda",
)

pipe_instruct = pipeline(
    "text-generation",
    model=model_instruct,
    tokenizer=tokenizer_instruct,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    # device="cuda",
)

pipe_dpo = pipeline(
    "text-generation",
    model=model_dpo,
    tokenizer=tokenizer_dpo,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    # device="cuda",
)

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


In [46]:
outputs= pipe(
    input,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1.0,
    top_p=0.95,
)

outputs_sft = pipe_sft(
    input,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1.0,
    top_p=0.95,
)

outputs_instruct = pipe_instruct(
    input,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1.0,
    top_p=0.95,
)

outputs_dpo = pipe_dpo(
    input,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1.0,
    top_p=0.95,
)

In [53]:
index = 50

input = ds["test"][index]["chosen"][:-1]

dash_line = "-".join("" for x in range(100))

print(dash_line)

print(tokenizer_dpo.apply_chat_template(input,tokenize=False))

print(dash_line)

dataset_chosen_response = ds["test"][index]["chosen"][-1:]

print(f"dataset_response (chosen):\n \n {tokenizer_dpo.apply_chat_template(dataset_chosen_response,tokenize=False)} ")

print(dash_line)

assistant_response = outputs[0]["generated_text"][-1]["content"]

print(f"Response by Llama3 before SFT:\n \n {assistant_response} ")

print(dash_line)

assistant_response_sft = outputs_sft[0]["generated_text"][-1]["content"]

print(f"Response by Llama3 after SFT: \n \n {assistant_response_sft}")

print(dash_line)

assistant_response_instruct = outputs_instruct[0]["generated_text"][-1]["content"]

print(f"Response by Llama3 Instruct: \n \n {assistant_response_instruct}")

print(dash_line)

assistant_response_dpo = outputs_dpo[0]["generated_text"][-1]["content"]

print(f"Response by Llama3 Instruct (after DPO): \n \n {assistant_response_dpo}")

---------------------------------------------------------------------------------------------------
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Teacher:In this task, you're given reviews from Amazon's food products and a summary of that review. Your task is to classify whether the given summary matches the original review. Generate "True" if the given review and its summary match, otherwise generate "False".
Teacher: Now, understand the problem? Solve this instance: Reviews: product was shipped in orginal box placed inside a larger box, no packing was used. Bulbs were broken when they arrived. 
 Summary: product was shipped in orginal box placed inside a larger ...
Student:<|eot_id|>
---------------------------------------------------------------------------------------------------
dataset_response (chosen):
 
 <|begin_of_text|><|start_header_id|>assistant<|end_header_id|>

True<|eot_id|> 
----------------------------------------------------------------------------------

In [56]:
inputs = ds["test"][100:110]["chosen"]
len(inputs)

10

In [64]:
import pandas as pd
input_prompts = []
chosen_responses = []
llama3_sft_responses = []
llama3_instruct_responses = []
llama3_dpo_responses = []

for dialogue in inputs:
    chosen_responses.append(tokenizer_dpo.apply_chat_template(dialogue[-1:],tokenize=False))

    input = dialogue[:-1]

    input_prompts.append(tokenizer_dpo.apply_chat_template(dialogue[:-1],tokenize=False))

    outputs_sft = pipe(
        input,
        max_new_tokens=100,
        eos_token_id=terminators,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
    )

    outputs_instruct = pipe_instruct(
        input,
        max_new_tokens=100,
        eos_token_id=terminators,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
    )

    outputs_dpo = pipe_dpo(
        input,
        max_new_tokens=100,
        eos_token_id=terminators,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
    )
    
    llama3_sft_responses.append(outputs_sft[0]["generated_text"][-1]["content"])
    llama3_instruct_responses.append(outputs_instruct[0]["generated_text"][-1]["content"])
    llama3_dpo_responses.append(outputs_dpo[0]["generated_text"][-1]["content"])

all_examples = list(zip(input_prompts, chosen_responses, llama3_sft_responses, llama3_instruct_responses, llama3_dpo_responses))

df = pd.DataFrame(all_examples,columns=["input","chosen","sft","instruct","dpo"])
df

,input,chosen,sft,instruct,dpo
0,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,Equilibrium price and quantity for robots:\nQ ...,**Equilibrium Price and Quantity for Each Toy:...,To find the equilibrium price and quantity for...
1,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,"The question asks about a specific operation, ...","A: During his time in the United States, he wr...",A: When was the subject of White Lies born?\nP...
2,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,"- Upon clicking the button, a modal window sho...",Here are the instructions to create a Lightnin...,Here are the steps to create a lightning compo...
3,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,Code-breaking played a crucial role in the Bat...,Code-breaking played a crucial role in the Bat...,Code-breaking played a crucial role in the Bat...
4,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,"I do not have access to visual images. But, I ...","The EE Buck, HIL, and test bench are all conce...","EE Buck, HIL, and test bench are all terms use..."
5,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,Online courses can provide individuals seeking...,Online courses can be a valuable resource for ...,Online courses can be a valuable resource for ...
6,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,Acknowledged.,Acknowledged,Acknowledged
7,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,"Sure, let's start an AI project! What is the o...",I'd be happy to start an AI project with you. ...,Exciting! I'd be delighted to help you start a...
8,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,Diagnosis: Anxiety disorder\nTreatment plan: I...,**Diagnosis:** Social Anxiety Disorder (SAD)\n...,Diagnosis: Social Anxiety Disorder (SAD)\n\nTr...
9,<|begin_of_text|><|start_header_id|>user<|end_...,<|begin_of_text|><|start_header_id|>assistant<...,"Dear Chretien,\n\nI am glad to hear that we ha...",Here is a reviewed and corrected version of th...,Here is a revised version of the email that in...
